<a href="https://colab.research.google.com/github/dev-AlexandreMoreira/da-ebac-alexandremoreira/blob/main/An%C3%A1lise_de_dados_Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Aula**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>

---

# **Aulas**

## 1\. Introdução

### **1.1. TLDR**

 - **Dashboard**:
  - Google Data Studio ([link](https://lookerstudio.google.com/reporting/3325b1e5-57bc-4ae3-8ae9-3d2601877cc3)).
 - **Processamento**:
  - Kaggle Notebook (`link`).
 - **Fontes**:
  - Casos pelo site do Ministério da Saúde ([link](https://covid.saude.gov.br/));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

### **1.2. Pandemia Coronavírus 2020**

> A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil. O processamento de dados está neste `link` e o dashboard, neste ([link](https://lookerstudio.google.com/s/lWni_vNdlsQ)).

### **1.3. Dados**

Os dados sobre **casos da COVID-19** são compilados pelo site do Ministério da Saúde ([link](https://covid.saude.gov.br/)). Os dados são atualizados diariamente desde o início da pandemia e conta com uma granularidade temporal de dias e geográfica de regiões do país (estados, municípios, etc.).

Os dados sobre **vacinação da COVID-19** são compilados pelo projeto Nosso Mundo em Dados (*Our World in Data* ou OWID) da universidade britânica de **Oxford** ([link](https://www.ox.ac.uk)). Os dados são **atualizados diariamente** deste janeiro de 2020 com uma **granularidade temporal de dias e geográfica de países**. O website do projeto pode ser acessado neste [link](https://ourworldindata.org) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## 2\. Análise Exploratória de Dados

Nesta sessão vamos utilizar os seguintes pacotes Python para processar os dados bruto em um formato adequado para um painel para exploração interativa de dados.

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

### **2.1. Casos**

Vamos trabalhar com os dados de **casos** do Ministério da Saúde considerando o perído de Janeiro a Dezembro de 2021, que foi o período mais critico de casos e óbitos no nosso país.



#### **2.1.1. Extração**

Os dados disponibilizados pelo site do ministério da saúde já se encontram bem estruturados e compilado em formato .csv

#### **2.1.2. Wrangling**

Vamos manipular os dados para o dashboard. O foco é em garantir uma boa granularidade e qualidade da base de dados. Dado o volume de informação contida no perído o site do ministéria da saúde disponibiliza os dados em duas partes, sendo assim, iremos carrega-los e em seguida concatena-lo em um único dataframe.

In [2]:
import pandas as pd

# Carregar arquivo1
df_arquivo1 = pd.read_csv('COVIDBR_2021_Parte1.csv', sep=';')

# Carregar arquivo2
df_arquivo2 = pd.read_csv('COVIDBR_2021_Parte2.csv', sep=';')

#concatenação dos arquivos
casos_covid = pd.concat([df_arquivo1, df_arquivo2], ignore_index=True)


In [3]:
casos_covid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-01,53,210147125.0,7700578.0,24605,195411,462,6747065.0,733959.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-02,53,210147125.0,7716405.0,15827,195725,314,6756284.0,748883.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-03,1,210147125.0,7733746.0,17341,196018,293,6769420.0,751260.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-04,1,210147125.0,7753752.0,20006,196561,543,6813008.0,724720.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-01-05,1,210147125.0,7810400.0,56648,197732,1171,6875230.0,681961.0,NaN


In [4]:
casos_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050935 entries, 0 to 2050934
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   regiao                  object 
 1   estado                  object 
 2   municipio               object 
 3   coduf                   int64  
 4   codmun                  float64
 5   codRegiaoSaude          float64
 6   nomeRegiaoSaude         object 
 7   data                    object 
 8   semanaEpi               int64  
 9   populacaoTCU2019        float64
 10  casosAcumulado          float64
 11  casosNovos              int64  
 12  obitosAcumulado         int64  
 13  obitosNovos             int64  
 14  Recuperadosnovos        float64
 15  emAcompanhamentoNovos   float64
 16  interior/metropolitana  float64
dtypes: float64(7), int64(5), object(5)
memory usage: 266.0+ MB


Tratamento dos dados

In [5]:
import pandas as pd

# Remover colunas indesejadas
casos_covid = casos_covid.drop(columns=['nomeRegiaoSaude', 'semanaEpi', 'emAcompanhamentoNovos', 'interior/metropolitana','coduf','codmun','codRegiaoSaude','Recuperadosnovos'])

# Excluir linhas com qualquer valor ausente (NaN)
casos_covid = casos_covid.dropna()


In [6]:
#Tratamento de valores faltantes e negativos em colunas específicas

# Identifique colunas para substituir valores negativos
colunas_valores_negativos = ['casosNovos', 'obitosNovos', 'casosAcumulado', 'obitosAcumulado']

# Substitua valores negativos por 0 usando atribuição com máscara
# Crie uma máscara que indica True para valores não negativos e False para valores negativos
mascara = casos_covid[colunas_valores_negativos] >= 0

# Substitua os valores negativos por 0 na máscara
casos_covid.update(casos_covid[colunas_valores_negativos].where(mascara, 0))



In [7]:
#Converter campo tipo data para formato BR
casos_covid['data'] = pd.to_datetime(casos_covid['data'], format='%Y-%m-%d')

#Converter campo municipio para string e eliminar espaços em branco
casos_covid['municipio'] = casos_covid['municipio'].str.strip().str.lower()

#Ordenar o dataframe por municipio/data
casos_covid = casos_covid.sort_values(by=[ 'municipio','data'], ascending=True)

In [8]:
# Substituir as siglas por nomes completos na coluna "estado"
# Criando o dicionário sigla-estado
sigla_estado = {
    'AC' : 'Acre',
    'AL' : 'Alagoas',
    'AP' : 'Amapá',
    'AM' : 'Amazonas',
    'BA' : 'Bahia',
    'CE' : 'Ceará',
    'ES' : 'Espírito Santo',
    'GO' : 'Goiás',
    'MA' : 'Maranhão',
    'MT' : 'Mato Grosso',
    'MS' : 'Mato Grosso do Sul',
    'MG' : 'Minas Gerais',
    'PA' : 'Pará',
    'PB' : 'Paraíba',
    'PR' : 'Paraná',
    'PE' : 'Pernambuco',
    'PI' : 'Piauí',
    'RJ' : 'Rio de Janeiro',
    'RN' : 'Rio Grande do Norte',
    'RS' : 'Rio Grande do Sul',
    'RO' : 'Rondônia',
    'RR' : 'Roraima',
    'SC' : 'Santa Catarina',
    'SP' : 'São Paulo',
    'SE' : 'Sergipe',
    'TO' : 'Tocantins',
    'DF' : 'Distrito Federal'
}

casos_covid['estado'] = casos_covid['estado'].replace(sigla_estado)

In [9]:
#Renomeando colunas
casos_covid = casos_covid.rename(
  columns={
    'populacaoTCU2019': 'Populacao'
  }
)

for col in casos_covid.columns:
  casos_covid = casos_covid.rename(columns={col: col.lower()})

In [10]:
#Filtrando somente municípios com mais de 100.000 habitantes
casos_covid = casos_covid[casos_covid['populacao'] > 100000]

In [11]:
#Garantindo que não haja duplicidade de registros para um mesma data/município
import pandas as pd

# Group by 'casos_covid' por 'data' and 'municipio'
casos_covid = casos_covid.groupby(['municipio','data'])

# Dropando as duplicidades
casos_covid = casos_covid.first().reset_index()

# Dropando linhas com valores faltantes nas colunas 'casosacumulado', 'obitosacumulado'
casos_covid = casos_covid.dropna(subset=['casosacumulado', 'obitosacumulado'])

# Dropando linhas com valores negativos nas colunas 'casosNovos' e 'obitosNovos'
casos_covid = casos_covid.drop(
    index=casos_covid[(casos_covid['casosnovos'] < 0) | (casos_covid['obitosnovos'] < 0)].index
)

In [12]:
#Criando colunas temporais
# Convertendo 'data' para datetime
casos_covid['data'] = pd.to_datetime(casos_covid['data'], format='%Y-%m-%d')

# Criando colunas 'mes' e 'ano'
casos_covid['mes'] = casos_covid['data'].dt.strftime('%m-%y')
casos_covid['ano'] = casos_covid['data'].dt.strftime('%Y')


In [13]:
casos_covid.head()

,municipio,data,regiao,estado,populacao,casosacumulado,casosnovos,obitosacumulado,obitosnovos,mes,ano
0,abaetetuba,2021-01-01,Norte,Pará,157698.0,5120.0,1,121,1,01-21,2021
1,abaetetuba,2021-01-02,Norte,Pará,157698.0,5120.0,0,121,0,01-21,2021
2,abaetetuba,2021-01-03,Norte,Pará,157698.0,5121.0,1,122,1,01-21,2021
3,abaetetuba,2021-01-04,Norte,Pará,157698.0,5121.0,0,122,0,01-21,2021
4,abaetetuba,2021-01-05,Norte,Pará,157698.0,5137.0,16,122,0,01-21,2021


In [14]:
casos_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118260 entries, 0 to 118259
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   municipio        118260 non-null  object        
 1   data             118260 non-null  datetime64[ns]
 2   regiao           118260 non-null  object        
 3   estado           118260 non-null  object        
 4   populacao        118260 non-null  float64       
 5   casosacumulado   118260 non-null  float64       
 6   casosnovos       118260 non-null  int64         
 7   obitosacumulado  118260 non-null  int64         
 8   obitosnovos      118260 non-null  int64         
 9   mes              118260 non-null  object        
 10  ano              118260 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 9.9+ MB


In [15]:
casos_covid.to_csv('./covid_arq_tratado.csv', sep=',', index=False)

In [16]:
casos_covid = pd.read_csv('covid_arq_tratado.csv', sep=',')

In [17]:
casos_covid.head()

,municipio,data,regiao,estado,populacao,casosacumulado,casosnovos,obitosacumulado,obitosnovos,mes,ano
0,abaetetuba,2021-01-01,Norte,Pará,157698.0,5120.0,1,121,1,01-21,2021
1,abaetetuba,2021-01-02,Norte,Pará,157698.0,5120.0,0,121,0,01-21,2021
2,abaetetuba,2021-01-03,Norte,Pará,157698.0,5121.0,1,122,1,01-21,2021
3,abaetetuba,2021-01-04,Norte,Pará,157698.0,5121.0,0,122,0,01-21,2021
4,abaetetuba,2021-01-05,Norte,Pará,157698.0,5137.0,16,122,0,01-21,2021


 - Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:

Separação em Blocos:

Para facilitar a leitura e compreensão do código, foi escrito em blocos menores, com comentários explicativos para cada etapa:

1. Cálculo da Média Móvel (7 dias) de Casos e Óbitos

In [18]:
import pandas as pd

# Calculando a média móvel de 7 dias para casos acumulados
casos_covid['casos_avg7'] = casos_covid['casosacumulado'].rolling(window=7).mean()

# Calculando a média móvel de 7 dias para óbitos acumulados
casos_covid['obitos_avg7'] = casos_covid['obitosacumulado'].rolling(window=7).mean()


2. Cálculo da variação percentual de Casos e Óbitos:



In [19]:
# Calculando a variação percentual da média móvel de 7 dias para casos acumulados (últimos 14 dias)
casos_covid['var_casos_7d'] = casos_covid['casos_avg7'].pct_change() * 100

# Calculando a variação percentual da média móvel de 7 dias para óbitos acumulados (últimos 14 dias)
casos_covid['var_obitos_7d'] = casos_covid['obitos_avg7'].pct_change() * 100



3. Criando colunas para indicar a estabilidade

In [20]:
import pandas as pd

# Calculando a média móvel de 7 dias para casos acumulados
casos_covid['casos_avg7'] = casos_covid['casosacumulado'].rolling(window=7).mean().round(2)

# Calculando a média móvel de 7 dias para óbitos acumulados
casos_covid['obitos_avg7'] = casos_covid['obitosacumulado'].rolling(window=7).mean().round(2)

# Calculando a variação percentual da média móvel de 7 dias para casos acumulados (últimos 14 dias)
casos_covid['var_casos_7d'] = round(casos_covid['casos_avg7'].pct_change() * 100, 2)

# Calculando a variação percentual da média móvel de 7 dias para óbitos acumulados (últimos 14 dias)
casos_covid['var_obitos_7d'] = round(casos_covid['obitos_avg7'].pct_change() * 100, 2)

# Definir bordas (bin edges) para categorizar a variação de casos em 4 categorias
bordas_casos = [-np.inf, -15, -5, 5, 15, np.inf]  # Ajuste as bordas para as categorias desejadas

# Definir rótulos (labels) para as 4 categorias de variação de casos
rotulos_casos = ['baixa', 'média', 'alta', 'muito alta', 'extremamente alta']  # Add one more label

# Criar a coluna "estabilidade_casos" com categorização por bordas e rótulos
casos_covid['estabilidade_casos'] = pd.cut(casos_covid['var_casos_7d'], bordas_casos, labels=rotulos_casos)


In [21]:
casos_covid.head(15)

,municipio,data,regiao,estado,populacao,casosacumulado,casosnovos,obitosacumulado,obitosnovos,mes,ano,casos_avg7,obitos_avg7,var_casos_7d,var_obitos_7d,estabilidade_casos
0,abaetetuba,2021-01-01,Norte,Pará,157698.0,5120.0,1,121,1,01-21,2021,NaN,NaN,NaN,NaN,NaN
1,abaetetuba,2021-01-02,Norte,Pará,157698.0,5120.0,0,121,0,01-21,2021,NaN,NaN,NaN,NaN,NaN
2,abaetetuba,2021-01-03,Norte,Pará,157698.0,5121.0,1,122,1,01-21,2021,NaN,NaN,NaN,NaN,NaN
3,abaetetuba,2021-01-04,Norte,Pará,157698.0,5121.0,0,122,0,01-21,2021,NaN,NaN,NaN,NaN,NaN
4,abaetetuba,2021-01-05,Norte,Pará,157698.0,5137.0,16,122,0,01-21,2021,NaN,NaN,NaN,NaN,NaN
5,abaetetuba,2021-01-06,Norte,Pará,157698.0,5138.0,1,122,0,01-21,2021,NaN,NaN,NaN,NaN,NaN
6,abaetetuba,2021-01-07,Norte,Pará,157698.0,5340.0,202,122,0,01-21,2021,5156.71,121.71,NaN,NaN,NaN
7,abaetetuba,2021-01-08,Norte,Pará,157698.0,5372.0,32,122,0,01-21,2021,5192.71,121.86,0.70,0.12,alta
8,abaetetuba,2021-01-09,Norte,Pará,157698.0,5381.0,9,122,0,01-21,2021,5230.00,122.00,0.72,0.11,alta
9,abaetetuba,2021-01-10,Norte,Pará,157698.0,5381.0,0,122,0,01-21,2021,5267.14,122.00,0.71,0.00,alta


| 1 | 2 | 3 | 4 | 5 | 6 | <font color='red'>7</font> | <font color='green'>8</font> | 9 | 10 | 11 | 12 | 13 | <font color='blue'>14<font color='red'> | 15 | 16 | 17 | 18 | 19 | 20 | 21 |
| - | - | - | - | - | - | - | - | - | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| <font color='red'>D-6</font> | <font color='red'>D-5</font> | <font color='red'>D-4</font> | <font color='red'>D-3</font> | <font color='red'>D-2</font> | <font color='red'>D-1</font> | <font color='red'>D0</font> | | | | | | | | | | | | | | |
| D-7 | <font color='green'>D-6</font> | <font color='green'>D-5</font> | <font color='green'>D-4</font> | <font color='green'>D-3</font> | <font color='green'>D-2</font> | <font color='green'>D-1</font> | <font color='green'>D0</font> | | | | | | | | | | | | | |
| D-13 | D-12 | D-11 | D-10 | D-9 | D-8 | D-7 | <font color='blue'>D-6</font> | <font color='blue'>D-5</font> | <font color='blue'>D-4</font> | <font color='blue'>D-3</font> | <font color='blue'>D-2</font> | <font color='blue'>D-1</font> | <font color='blue'>D0</font> | | | | | | | |

In [22]:
casos_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118260 entries, 0 to 118259
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   municipio           118260 non-null  object  
 1   data                118260 non-null  object  
 2   regiao              118260 non-null  object  
 3   estado              118260 non-null  object  
 4   populacao           118260 non-null  float64 
 5   casosacumulado      118260 non-null  float64 
 6   casosnovos          118260 non-null  int64   
 7   obitosacumulado     118260 non-null  int64   
 8   obitosnovos         118260 non-null  int64   
 9   mes                 118260 non-null  object  
 10  ano                 118260 non-null  int64   
 11  casos_avg7          118254 non-null  float64 
 12  obitos_avg7         118254 non-null  float64 
 13  var_casos_7d        118253 non-null  float64 
 14  var_obitos_7d       118253 non-null  float64 
 15  estabilidade_caso

#### **2.1.3. Carregamento**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [23]:
casos_covid.to_csv('./covid-casos-2021.csv', sep=',', index=False)

### **2.2. Vacinação**

Vamos processar os dados de **vacinação** da universidade de Oxford.

#### **2.2.1. Extração**

Os dados estão compilados em um único arquivo.

In [24]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-24-608e6d2dd56d>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [25]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


Vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

In [26]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [27]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-09


#### **2.2.2. Wrangling**

Vamos manipular os dados para o dashboard. O foco é em garantir uma boa granularidade e qualidade da base de dados.

In [28]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-09


In [29]:
vaccines.shape

(1576, 7)

In [30]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1576 entries, 0 to 1575
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1576 non-null   object        
 1   population               1576 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1576 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 86.3+ KB


Vamos começar tratando os dados faltantes, a estratégia será a de preencher os buracos com o valor anterior válido mais próximo.

In [31]:
vaccines = vaccines.fillna(method='ffill')

Vamos também filtrar a base de dados de acordo com a coluna `date` para garantir que ambas as bases de dados tratam do mesmo período de tempo.

In [32]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

Agora, vamos alterar o nome das colunas.

In [32]:
vaccines = vaccines.rename(
  columns={
    'location': 'pais',
    'total_vaccinations': 'total_vacinados',
    'people_vaccinated': 'dose1',
    'people_fully_vaccinated': 'dose2',
    'total_boosters': 'dose3',
  }
)

Vamos então computar novas colunas para enriquecer a base de dados.

 - Chaves temporais:

In [33]:
vaccines['mes_vacinacao'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['ano_vacinacao']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

 - Dados relativos:

In [34]:
vaccines['dose1_perc'] = round(vaccines['dose1'] / vaccines['population'], 4)
vaccines['dose2_perc'] = round(vaccines['dose2'] / vaccines['population'], 4)
vaccines['dose3_perc'] = round(vaccines['dose3'] / vaccines['population'], 4)

Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [35]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total_vacinados'] = vaccines['total_vacinados'].astype('Int64')
vaccines['dose1'] = vaccines['dose1'].astype('Int64')
vaccines['dose2'] = vaccines['dose2'].astype('Int64')
vaccines['dose3'] = vaccines['dose3'].astype('Int64')

Por fim, vamos reorganizar as colunas e conferir o resultado final.

In [36]:
vaccines = vaccines[['date', 'pais', 'population', 'total_vacinados', 'dose1', 'dose1_perc', 'dose2', 'dose2_perc', 'dose3', 'dose3_perc', 'mes_vacinacao', 'ano_vacinacao']]

In [37]:
vaccines.tail()

,date,pais,population,total_vacinados,dose1,dose1_perc,dose2,dose2_perc,dose3,dose3_perc,mes_vacinacao,ano_vacinacao
1571,2024-04-24,Brazil,215313504,486436436,189643431,0.8808,176164186,0.8182,126388587,0.587,2024-04,2024
1572,2024-04-25,Brazil,215313504,486436436,189643431,0.8808,176164186,0.8182,126388587,0.587,2024-04,2024
1573,2024-04-26,Brazil,215313504,486436436,189643431,0.8808,176164186,0.8182,126388587,0.587,2024-04,2024
1574,2024-04-27,Brazil,215313504,486436436,189643431,0.8808,176164186,0.8182,126388587,0.587,2024-04,2024
1575,2024-04-28,Brazil,215313504,486436436,189643431,0.8808,176164186,0.8182,126388587,0.587,2024-04,2024


#### **2.2.3. Carregamento**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [38]:
vaccines.to_csv('./covid-vacinacao.csv', sep=',', index=False)

## 3\. Exploração Interativa de Dados

### **3.1. KPIs**

O dashboard de dados contem os seguintes indicadores chaves de desempenho (*key performance indicator* ou KPI) consolidados:

1. Casos e mortes nas 24 horas;
1. Média móvel (7 dias) de casos e mortes;
1. Tendência de casos e mortes;
1. Proporção de vacinados com 1ª, 2ª e 3ª doses.

### **3.2. EDA**

O dashboard de dados contem os seguintes gráficos para a análise exploratória de dados (*exploratory data analysis*
ou EDA) interativa:

1. Distribuição do números de casos e mortes ao longo do tempo;
1. Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;
1. Distribuição geográfica dos casos por estado por dia.

## 4\. Storytelling